In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# modelling
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [5]:
df = pd.read_csv('data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [6]:
X = df.drop(columns=['math_score'])

In [10]:
y = df['math_score']

In [13]:
# Create column Transformer with 3 types of trnasformers
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)

X = preprocessor.fit_transform(X)

In [20]:
# separate dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape


((800, 19), (200, 19))

In [21]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)    
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [30]:
models = {
    "Linear_Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "KNeighbors_Regressor": KNeighborsRegressor(),
    "Decision_Tree": DecisionTreeRegressor(),
    "Random_Forest_Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting_Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost_Regressor": AdaBoostRegressor()
}
model_list = []
r2_list = []

for k, v in models.items():
    model = v
    model.fit(X_train, y_train) # Train model
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    model_list.append(k)
    r2_list.append(model_test_r2)
    
    print("[ ", k, " ]")
    
    print("Model performance for Training set")
    print(f"- Root Mean Squared Error: {model_train_rmse:.4f}")
    print(f"- Mean Absolute Error: {model_train_mae:.4f}")
    print(f"- R2 Score: {model_train_r2:.4f}")
    
    print("---------------------------")
    
    print("Model performance for Test set")
    print(f"- Root Mean Squared Error: {model_test_rmse:.4f}")
    print(f"- Mean Absolute Error: {model_test_mae:.4f}")
    print(f"- R2 Score: {model_test_r2:.4f}")
    
    print("=================================", end='\n\n')
    
    

[  Linear_Regression  ]
Model performance for Training set
- Root Mean Squared Error: 5.3253
- Mean Absolute Error: 4.2698
- R2 Score: 0.8742
---------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4242
- Mean Absolute Error: 4.2236
- R2 Score: 0.8791

[  Lasso  ]
Model performance for Training set
- Root Mean Squared Error: 6.5938
- Mean Absolute Error: 5.2063
- R2 Score: 0.8071
---------------------------
Model performance for Test set
- Root Mean Squared Error: 6.5197
- Mean Absolute Error: 5.1579
- R2 Score: 0.8253

[  Ridge  ]
Model performance for Training set
- Root Mean Squared Error: 5.3233
- Mean Absolute Error: 4.2650
- R2 Score: 0.8743
---------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3904
- Mean Absolute Error: 4.2111
- R2 Score: 0.8806

[  KNeighbors_Regressor  ]
Model performance for Training set
- Root Mean Squared Error: 5.7077
- Mean Absolute Error: 4.5167
- R2 Score: 0.8555
------------------------

## Results

In [38]:
pd.DataFrame(zip(model_list, r2_list))

,0,1
0,Linear_Regression,0.879090
1,Lasso,0.825320
2,Ridge,0.880593
3,KNeighbors_Regressor,0.783813
4,Decision_Tree,0.754293
5,Random_Forest_Regressor,0.850245
6,XGBRegressor,0.827797
7,CatBoosting_Regressor,0.851632
8,AdaBoost_Regressor,0.854484


In [39]:
res_df = pd.DataFrame(zip(model_list, r2_list), columns=['Model_name', 'R2_score']).sort_values(by='R2_score', ascending=False)
res_df

,Model_name,R2_score
2,Ridge,0.880593
0,Linear_Regression,0.879090
8,AdaBoost_Regressor,0.854484
7,CatBoosting_Regressor,0.851632
5,Random_Forest_Regressor,0.850245
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,KNeighbors_Regressor,0.783813
4,Decision_Tree,0.754293
